In [2]:
! pip install seaborn
! pip install astral
! pip install swifter
! pip install scikit-learn-intelex

In [3]:
import warnings
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearnex import patch_sklearn, config_context
patch_sklearn()
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [8]:
import os
ipynb_path = os.getcwd()
src_path = os.path.join(ipynb_path, '/src/')
input_path = os.path.join(ipynb_path,"input/")

In [9]:
src_path

'/src/'

In [10]:
!pip install windpowerlib

In [15]:

import warnings
import os
import sys

import scipy.stats as spst

sys.path.append(src_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
import dask.dataframe as dd
from windpowerlib.wind_speed import logarithmic_profile
from src.utils import uv_to_wsd

In [166]:
os.getcwd()

'/home/user/Wind_Power'

In [167]:
input_path

'/home/user/Wind_Power/input/'

In [168]:
!pip install pyarrow
!pip install fastparquet


In [16]:
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

print("Power: ", power.shape)
print("train_y: ", gj_y.shape)
print("LDAPS: ", ldaps.shape)

Power:  (155528, 29)
train_y:  (52608, 4)
LDAPS:  (235818, 15)


In [17]:
!pip install xgboost

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

# yongmin's functions
from src.utils import DataConnector
from src.metric import NMAE
from src.data_processor import *

# model import
import xgboost as xgb
from xgboost import XGBRegressor

## logging
#import mlflow
#import mlflow.sklearn
#import mlflow.xgboost
#from mlflow.models import infer_signature

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [19]:
# 데이터 로드
power_2020 = pd.read_parquet(input_path + "dynamic_report_ewp02_2020_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2021 = pd.read_parquet(input_path + "dynamic_report_ewp02_2021_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]
power_2022 = pd.read_parquet(input_path + "dynamic_report_ewp02_2022_10min.parquet").rename({'Date/Time': 'dt', 'WTG.Name': 'turbine_id'}, axis=1)[:-3]

# 데이터 합치기
power = pd.concat([power_2020, power_2021, power_2022], ignore_index=True)

# 추가 데이터 로드
gj_y = pd.read_parquet(input_path + "train_y.parquet").rename({'end_datetime': 'dt'}, axis=1)
ldaps = pd.read_parquet(input_path + "train_ldaps_gyeongju.parquet")

In [20]:
# 파이프라인 구성 및 적용
DataPipeline = Pipeline([
    ('uv_transform', UVTransformer('wind_u_10m', 'wind_v_10m')),
    ('wind_transform', WindTransformer('wind_speed', 10, 100, ldaps['surf_rough'].mean())),
    ('feature_engineering', FeatureTransformer()),
])

# 파이프라인을 이용하여 ldaps 데이터 변환
ldaps_transformed = DataPipeline.fit_transform(ldaps)

print(ldaps_transformed.shape)


(235818, 23)


In [21]:
ldaps_transformed['turbine_id'].value_counts()

turbine_id
WTG01    26202
WTG02    26202
WTG03    26202
WTG04    26202
WTG05    26202
WTG06    26202
WTG07    26202
WTG08    26202
WTG09    26202
Name: count, dtype: int64[pyarrow]

In [22]:
pivot_ldaps = ldaps_transformed.pivot_table(index='dt', columns='turbine_id', aggfunc='mean')

pivot_ldaps.columns = ['_'.join(map(str, col)) for col in pivot_ldaps.columns]

In [23]:
pivot_ldaps.shape

(26202, 189)

In [24]:
average_ldaps = ldaps_transformed.drop('turbine_id', axis=1).groupby('dt').mean()
average_ldaps.shape

(26202, 21)

In [25]:
pivot_ldaps.columns = pivot_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
pivot_ldaps.columns = pivot_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
pivot_ldaps.columns = pivot_ldaps.columns.str.replace(r'__+', '_', regex=True)

In [26]:
average_ldaps.columns = average_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
average_ldaps.columns = average_ldaps.columns.str.replace(r'__+', '_', regex=True)

In [27]:
average_ldaps.reset_index(inplace=True)
pivot_ldaps.reset_index(inplace=True)

In [28]:
pivot_ldaps.head()

,dt,Night_WTG01,Night_WTG02,Night_WTG03,Night_WTG04,Night_WTG05,Night_WTG06,Night_WTG07,Night_WTG08,Night_WTG09,...,wind_u_10m_WTG09,wind_v_10m_WTG01,wind_v_10m_WTG02,wind_v_10m_WTG03,wind_v_10m_WTG04,wind_v_10m_WTG05,wind_v_10m_WTG06,wind_v_10m_WTG07,wind_v_10m_WTG08,wind_v_10m_WTG09
0,2020-01-02 00:00:00+09:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.510981,-2.640615,-2.904043,-3.657705,-3.657705,-3.657705,-3.657705,-3.657705,-3.893301,-3.893301
1,2020-01-02 01:00:00+09:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.472733,-2.414076,-2.657606,-3.377577,-3.377577,-3.377577,-3.377577,-3.377577,-3.598768,-3.598768
2,2020-01-02 02:00:00+09:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.619642,-2.259913,-2.546779,-3.167872,-3.167872,-3.167872,-3.167872,-3.167872,-3.375392,-3.375392
3,2020-01-02 03:00:00+09:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.661708,-2.280371,-2.363501,-2.749731,-2.749731,-2.749731,-2.749731,-2.749731,-2.837622,-2.837622
4,2020-01-02 04:00:00+09:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.218147,-1.920128,-1.876671,-2.241783,-2.241783,-2.241783,-2.241783,-2.241783,-2.367027,-2.367027


In [29]:
pivot_ldaps['dt'] = pd.to_datetime(pivot_ldaps['dt']).dt.tz_localize(None)
gj_y['dt'] = pd.to_datetime(gj_y['dt']).dt.tz_localize(None)
average_ldaps['dt'] = pd.to_datetime(pivot_ldaps['dt']).dt.tz_localize(None)
gj_y['dt'] = pd.to_datetime(gj_y['dt']).dt.tz_localize(None)

pivot_data = pd.merge(pivot_ldaps, gj_y, on='dt', how='inner')
avg_data = pd.merge(average_ldaps, gj_y, on='dt', how='inner')

In [30]:
# 1. 모든 그룹에서 0이 아닌 값을 우선 선택하되, 모든 값이 0인 경우는 그대로 유지
# 그룹화하여 0이 아닌 값을 먼저 선택
pivot_data_sorted = pivot_data.sort_values(['dt', 'plant_name', 'energy_kwh'], ascending=[True, True, False])

# 2. 중복된 'dt'와 'plant_name'을 기준으로 중복 제거 (0이 아닌 값을 우선 유지하되, 모두 0이면 그대로 유지)
pivot_data_cleaned = pivot_data_sorted.drop_duplicates(subset=['dt', 'plant_name'], keep='first')

# avg_data도 같은 방식으로 처리
avg_data_sorted = avg_data.sort_values(['dt', 'plant_name', 'energy_kwh'], ascending=[True, True, False])
avg_data_cleaned = avg_data_sorted.drop_duplicates(subset=['dt', 'plant_name'], keep='first')

pivot_data_cleaned = pivot_data.drop_duplicates(subset=['dt'], keep='first')

# avg_data에서도 동일한 방식 적용
avg_data_cleaned = avg_data.drop_duplicates(subset=['dt'], keep='first')
pivot_data = pivot_data_cleaned
avg_data = avg_data_cleaned


In [31]:
avg_data.tail()

,dt,elevation,land_cover,surf_rough,frictional_vmax_50m,frictional_vmin_50m,pressure,relative_humid,specific_humid,temp_air,...,wind_direction,wind_speed_100m,season,tke,wind_shear,turbulence_intensity,Night,plant_name,period_hours,energy_kwh
52394,2022-12-31 19:00:00,303.944458,1.0,0.303777,9.547421,8.806270,99002.343750,74.180206,0.002686,271.694977,...,295.021698,11.433529,1.0,21.619039,0.197260,0.629139,1.0,경주풍력,1,18488.281250
52396,2022-12-31 20:00:00,303.944458,1.0,0.303777,9.305802,8.890262,98989.156250,76.329315,0.002686,271.688263,...,300.120575,12.239202,1.0,20.861759,0.198635,0.743938,1.0,경주풍력,1,18394.531250
52398,2022-12-31 21:00:00,303.944458,1.0,0.303777,9.148444,8.486347,98985.757812,79.236092,0.002686,271.246460,...,300.463837,11.379596,1.0,19.183256,0.197092,0.621454,1.0,경주풍력,1,18443.359375
52400,2022-12-31 22:00:00,303.944458,1.0,0.303777,8.522839,6.772115,98991.593750,82.242607,0.002686,270.194183,...,301.581329,8.629054,1.0,16.335480,0.190374,0.229535,1.0,경주풍력,1,18525.390625
52402,2022-12-31 23:00:00,303.944458,1.0,0.303777,6.764867,6.185848,98977.726562,79.409370,0.002512,269.774323,...,294.633240,7.457907,1.0,11.723340,0.186273,0.062661,1.0,경주풍력,1,18529.296875


In [32]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


def get_trasforms_datas(merged_data, numeric_columns, target):
    z_scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    
    x_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), numeric_columns]
    x_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), numeric_columns]

    y_train = merged_data.loc[merged_data['dt'].between('2020-01-01', '2020-12-31', inclusive='left'), target].shift(periods = -24)
    y_test = merged_data.loc[merged_data['dt'].between('2021-01-01', '2022-12-31', inclusive='left'), target].shift(periods = -24)
    #y_train = y_train.dropna()
    #y_test = y_test.dropna()

    x_train = x_train.iloc[:-24]
    y_train = y_train.iloc[:-24]

    x_test = x_test.iloc[:-24]
    y_test = y_test.iloc[:-24]

    # Min-Max Scaling
    x_train_m = minmax_scaler.fit_transform(x_train)
    x_train_m = pd.DataFrame(x_train_m, columns=x_train.columns)
    x_test_m = minmax_scaler.transform(x_test)
    x_test_m = pd.DataFrame(x_test_m, columns=x_train.columns)

    # Standard Scaling
    x_train_z = z_scaler.fit_transform(x_train)
    x_train_z = pd.DataFrame(x_train_z, columns=x_train.columns)
    x_test_z = z_scaler.transform(x_test)
    x_test_z = pd.DataFrame(x_test_z, columns=x_train.columns)

    return x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test

In [186]:
!pip install scikit-learn-extra

In [40]:
# for avg or unity data
#from sklearn.cluster import KMeans

## 이제 특징 생성에 클러스터링 결과는 보지 않을 예정.
#def addKmeansFeature(train_data, test_data):
#    pd.options.mode.chained_assignment = None

#    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
#        kmeans = KMeans(n_clusters=n_clusters, n_init=10)

#        train_data[f'cluster_{n_clusters}'] = kmeans.fit_predict(train_data[['wind_speed', 'wind_direction']])
        
#        test_data[f'cluster_{n_clusters}'] = kmeans.predict(test_data[['wind_speed', 'wind_direction']])

#    return train_data, test_data
#from sklearn.decomposition import PCA

#def addPCAFeature(train_data, test_data):
#    # PCA 적용할 특징 열 선택 (u, v 성분)
#    wind_features = ['storm_u_5m', 'storm_v_5m', 'wind_u_10m', 'wind_v_10m', 
#                     'wind_speed', 'wind_direction']
    
#    # 훈련 데이터에서 PCA 학습
#    pca = PCA(n_components=2)
#    pca_train = pca.fit_transform(train_data[wind_features])
    
#    # 훈련 데이터에 주성분 추가
#    train_data['PC1'] = pca_train[:, 0]
#    train_data['PC2'] = pca_train[:, 1]
    
#    # 테스트 데이터에 PCA 적용
#    pca_test = pca.transform(test_data[wind_features])
#    test_data['PC1'] = pca_test[:, 0]
#    test_data['PC2'] = pca_test[:, 1]

#    # PCA 설명력 확인
#    explained_variance = pca.explained_variance_ratio_
#    print(f"PC1 설명력: {explained_variance[0]}")
#    print(f"PC2 설명력: {explained_variance[1]}")

#    return train_data, test_data

#from sklearn_extra.cluster import KMedoids

#def addKMedoidsFeature(train_data, test_data):
#    pd.options.mode.chained_assignment = None

#    for n_clusters in range(2, 7):  # 2부터 6까지 클러스터 생성
#        kmedoids = KMedoids(n_clusters=n_clusters, random_state=42)

#        # 훈련 데이터에 K-Medoids 클러스터링 적용
#        train_data[f'medoid_cluster_{n_clusters}'] = kmedoids.fit_predict(train_data[['wind_speed', 'wind_direction']])

#        # 테스트 데이터에 학습된 K-Medoids 모델 적용
#        test_data[f'medoid_cluster_{n_clusters}'] = kmedoids.predict(test_data[['wind_speed', 'wind_direction']])

#    return train_data, test_data

# =========================================
# for pivot
from sklearn.cluster import KMeans

def addKmeansFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    wind_speed_columns = [col for col in train_data.columns if col.startswith('wind_speed')]
    wind_direction_columns = [col for col in train_data.columns if col.startswith('wind_direction')]

    for wind_speed_col, wind_direction_col in zip(wind_speed_columns, wind_direction_columns):
        for n_clusters in range(2, 7):
            kmeans = KMeans(n_clusters=n_clusters, n_init=10)

            train_data[f'cluster_{n_clusters}_{wind_speed_col}'] = kmeans.fit_predict(train_data[[wind_speed_col, wind_direction_col]])
            test_data[f'cluster_{n_clusters}_{wind_speed_col}'] = kmeans.predict(test_data[[wind_speed_col, wind_direction_col]])

    return train_data, test_data

from sklearn.decomposition import PCA

def addPCAFeature(train_data, test_data):
    wind_speed_columns = [col for col in train_data.columns if col.startswith('wind_speed')]
    wind_direction_columns = [col for col in train_data.columns if col.startswith('wind_direction')]
    
    for wind_speed_col, wind_direction_col in zip(wind_speed_columns, wind_direction_columns):
        wind_features = [wind_speed_col, wind_direction_col]

        pca = PCA(n_components=2)
        pca_train = pca.fit_transform(train_data[wind_features])
        pca_test = pca.transform(test_data[wind_features])

        train_data[f'PC1_{wind_speed_col}'] = pca_train[:, 0]
        train_data[f'PC2_{wind_speed_col}'] = pca_train[:, 1]
        test_data[f'PC1_{wind_speed_col}'] = pca_test[:, 0]
        test_data[f'PC2_{wind_speed_col}'] = pca_test[:, 1]

        explained_variance = pca.explained_variance_ratio_
        print(f"{wind_speed_col}: PC1 설명력: {explained_variance[0]}, PC2 설명력: {explained_variance[1]}")

    return train_data, test_data

from sklearn_extra.cluster import KMedoids

def addKMedoidsFeature(train_data, test_data):
    pd.options.mode.chained_assignment = None

    wind_speed_columns = [col for col in train_data.columns if col.startswith('wind_speed')]
    wind_direction_columns = [col for col in train_data.columns if col.startswith('wind_direction')]

    for wind_speed_col, wind_direction_col in zip(wind_speed_columns, wind_direction_columns):
        for n_clusters in range(2, 7):
            kmedoids = KMedoids(n_clusters=n_clusters, random_state=42)

            train_data[f'medoid_cluster_{n_clusters}_{wind_speed_col}'] = kmedoids.fit_predict(train_data[[wind_speed_col, wind_direction_col]])
            test_data[f'medoid_cluster_{n_clusters}_{wind_speed_col}'] = kmedoids.predict(test_data[[wind_speed_col, wind_direction_col]])

    return train_data, test_data


In [41]:
numeric_columns = pivot_data.select_dtypes(include=['number']).columns.tolist()

In [42]:
# 일단은 avg에 대해서만 테스트

#x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test = get_trasforms_datas(avg_data, numeric_columns, 'energy_kwh')

# Next, pivot data
x_train, x_test, x_train_m, x_test_m, x_train_z, x_test_z, y_train, y_test = get_trasforms_datas(pivot_data, numeric_columns, 'energy_kwh')

In [43]:
x_train.columns

Index(['Night_WTG01', 'Night_WTG02', 'Night_WTG03', 'Night_WTG04',
       'Night_WTG05', 'Night_WTG06', 'Night_WTG07', 'Night_WTG08',
       'Night_WTG09', 'elevation_WTG01',
       ...
       'wind_v_10m_WTG02', 'wind_v_10m_WTG03', 'wind_v_10m_WTG04',
       'wind_v_10m_WTG05', 'wind_v_10m_WTG06', 'wind_v_10m_WTG07',
       'wind_v_10m_WTG08', 'wind_v_10m_WTG09', 'period_hours', 'energy_kwh'],
      dtype='object', length=191)

In [45]:
x_train, x_test = addKmeansFeature(x_train, x_test)
x_train_m, x_test_m = addKmeansFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKmeansFeature(x_train_z, x_test_z)
print('kmean 적용 완료')
x_train, x_test = addPCAFeature(x_train, x_test)
x_train_m, x_test_m = addPCAFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addPCAFeature(x_train_z, x_test_z)
print('pca 적용 완료')

x_train, x_test = addKMedoidsFeature(x_train, x_test)
x_train_m, x_test_m = addKMedoidsFeature(x_train_m, x_test_m)
x_train_z, x_test_z = addKMedoidsFeature(x_train_z, x_test_z)
print('kmedoid 적용 완료')


kmean 적용 완료
wind_speed_WTG01: PC1 설명력: 0.9994230270385742, PC2 설명력: 0.0005769270355813205
wind_speed_WTG02: PC1 설명력: 0.9995388388633728, PC2 설명력: 0.0004610519390553236
wind_speed_WTG03: PC1 설명력: 0.9995805025100708, PC2 설명력: 0.00041955013875849545
wind_speed_WTG04: PC1 설명력: 0.9995805025100708, PC2 설명력: 0.00041955013875849545
wind_speed_WTG05: PC1 설명력: 0.9995805025100708, PC2 설명력: 0.00041955013875849545
wind_speed_WTG06: PC1 설명력: 0.9995805025100708, PC2 설명력: 0.00041955013875849545
wind_speed_WTG07: PC1 설명력: 0.9995805025100708, PC2 설명력: 0.00041955013875849545
wind_speed_WTG08: PC1 설명력: 0.9996228218078613, PC2 설명력: 0.0003771248448174447
wind_speed_WTG09: PC1 설명력: 0.9996228218078613, PC2 설명력: 0.0003771248448174447
wind_speed_WTG01: PC1 설명력: 0.8665077364818168, PC2 설명력: 0.13349226351818322
wind_speed_WTG02: PC1 설명력: 0.8718077494890404, PC2 설명력: 0.12819225051095964
wind_speed_WTG03: PC1 설명력: 0.8836382355875765, PC2 설명력: 0.11636176441242348
wind_speed_WTG04: PC1 설명력: 0.8836382355875765, PC2 설명

In [46]:
x_dict = {
    # 원본 데이터만 사용
    'original': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m'],

    # PCA 추가
    'pca_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                 'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                 'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'PC1', 'PC2'],

    # 클러스터(2~6) 추가
    'cluster_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                     'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                     'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                     'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # PCA + 클러스터(2~6) 추가
    'pca_and_cluster': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                        'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                        'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                        'PC1', 'PC2', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6'],

    # 클러스터 개수에 따른 경우
    'cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_2'],
    
    'cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_3'],
    
    'cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_4'],
    
    'cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_5'],
    
    'cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_6'],

    # KMedoids 추가
    'kmedoids_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                      'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                      'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                      'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # PCA + KMedoids 추가
    'pca_and_kmedoids': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                         'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                         'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 
                         'PC1', 'PC2', 'medoid_cluster_2', 'medoid_cluster_3', 'medoid_cluster_4', 'medoid_cluster_5', 'medoid_cluster_6'],

    # KMedoids 클러스터 개수에 따른 경우
    'medoid_cluster_2_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_2'],

    'medoid_cluster_3_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_3'],

    'medoid_cluster_4_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_4'],

    'medoid_cluster_5_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_5'],

    'medoid_cluster_6_only': ['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                              'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                              'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'medoid_cluster_6']

}


In [192]:
! pip install lightgbm

In [193]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV


In [47]:
# 하이퍼 파라미터 그리드
param_grids = {
    'GradientBoostingRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'SVR': {'C': [0.1, 1], 'kernel': ['linear']},
    'RandomForestRegressor': {'n_estimators': [1, 200], 'max_depth': [10, 20, None]},
    'LinearRegression': {},  # LinearRegression은 기본 파라미터로 사용
    'ExtraTreesRegressor': {'n_estimators': [11, 200], 'max_depth': [10, 20, None]},
    'AdaBoostRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'DecisionTreeRegressor': {'max_depth': [10, 20, None]},
    'XGBRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'XGBRFRegressor': {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]},
    'LGBMRegressor' : {'n_estimators': [1, 100], 'learning_rate': [0.01, 0.1]}
}

In [48]:
def plot_feature_importance(model, x_col,x_shape, model_name, save_dir):
    save_dir = os.path.join(ipynb_path, save_dir)
    
    try:
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]

        plt.figure(figsize=(10, 6))
        plt.title(f"Feature Importance : {model_name}")
        plt.bar(range(x_shape[1]), importances[indices], align='center')
        plt.xticks(range(x_shape[1]), x_col[indices], rotation=90)
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, f"{model_name}_feature_importance.png"))
        plt.close()
    except AttributeError:
        if hasattr(model, 'coef_'):
            importances = np.abs(model.coef_)
            indices = np.argsort(importances)[::-1]

            plt.figure(figsize=(10, 6))
            plt.title(f"Coefficient : {model_name}")
            plt.bar(range(x_shape[1]), importances[indices], align='center')
            plt.xticks(range(x_shape[1]), x_col[indices], rotation=90)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, f"{model_name}_coefficient.png"))
            plt.close()
        else:
            print(f"{model_name} has no feature importances or coefficients.")

In [49]:
def plot_predict_actual(pred, actual ,model_name, save_dir):
    plt.figure(figsize=(10, 6))
    plt.title(f"Feature Importance : {model_name}")
    plt.xticks(rotation=90)
    plt.plot(list(pred), label='pred')
    plt.plot(list(actual), label='Actual')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"{model_name}_predict.png"))
    plt.close()

In [54]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib
import os
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor, XGBRFRegressor
from lightgbm import LGBMRegressor

def train(save_dir, x_train, y_train, x_test, y_test):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    log_file = open(os.path.join(save_dir, "model_training_log.txt"), "w")

    models = [
        GradientBoostingRegressor(),
        RandomForestRegressor(),
        LinearRegression(),
        ExtraTreesRegressor(),
        AdaBoostRegressor(),
        #DecisionTreeRegressor(),
        XGBRegressor(tree_method='gpu_hist'),
        XGBRFRegressor(tree_method='gpu_hist'),
        LGBMRegressor(device='gpu', verbose=-1),
    ]

    results = {
        'Model_Name': [],
        'R2_score': [],
        'RMSE': [],
        'NMAE': [],
        'Best_Params': [],
        'Predict' : []
    }

    for model in models:
        model_name_current = model.__class__.__name__

        param_grid = param_grids.get(model_name_current, {})

        search = RandomizedSearchCV(
            model,
            param_distributions=param_grid,
            cv=3, 
            n_jobs=-1,
            n_iter=5,
            scoring='neg_mean_absolute_error'  # MAE를 최적화 기준으로 설정
        )

        log_file.write(f"Searching best hyperparameters for {model_name_current}\n")
        search.fit(x_train, y_train)
        best_model = search.best_estimator_
        best_params = search.best_params_

        log_file.write(f"Training {model_name_current}\n")
        print(f"Training {model_name_current}\n")

        y_pred = best_model.predict(x_test)

        # 성능 평가
        r2 = r2_score(y_test, y_pred) * 100
        rmse_value = mean_squared_error(y_test, y_pred, squared=False)
        nmae_value = NMAE(y_test, y_pred)

        results['Model_Name'].append(model_name_current)
        results['R2_score'].append(r2)
        results['RMSE'].append(rmse_value)
        results['NMAE'].append(nmae_value)
        results['Best_Params'].append(best_params)
        results['Predict'].append(y_pred)

        # 모델 저장
        joblib.dump(search, os.path.join(save_dir, f'{model_name_current}.pkl'))
        
        plot_predict_actual(y_pred, y_test, model_name_current, save_dir)

        log_file.write(f"R2 Score: {r2:.2f}\n")
        log_file.write(f"RMSE: {rmse_value:.2f}\n")
        log_file.write(f"NMAE: {nmae_value:.4f}\n")
        log_file.write(f"Best Hyperparameters: {best_params}\n\n")

        print(f"R2 Score: {r2:.2f}\n")
        print(f"RMSE: {rmse_value:.2f}\n")
        print(f"NMAE: {nmae_value:.4f}\n")
        print(f"Best Hyperparameters: {best_params}\n\n")

        # Feature importance plotting
        plot_feature_importance(best_model, x_train.columns, x_train.shape, model_name_current, save_dir)

    # 결과 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(save_dir, "model_scores.csv"), index=False)

    log_file.close()

In [56]:
#for key in x_dict.keys():
#    # 필요한 컬럼만 선택해서 데이터셋 생성
    #x_train_selected = x_train[x_dict[key]]
    #x_test_selected = x_test[x_dict[key]]
    #x_train_m_selected = x_train_m[x_dict[key]]
    #x_test_m_selected = x_test_m[x_dict[key]]
    #x_train_z_selected = x_train_z[x_dict[key]]
    #x_test_z_selected = x_test_z[x_dict[key]]
x_train_selected = x_train
x_test_selected = x_test
x_train_m_selected = x_train_m
x_test_m_selected = x_test_m
x_train_z_selected = x_train_z
x_test_z_selected = x_test_z

# ====================
# 정규화 없는 데이터
# ====================
save_dir = os.path.join(ipynb_path, f'notebooks/test_pivot/pivotall_General_testBaseline/')

train(save_dir, x_train_selected, y_train, x_test_selected, y_test)
# ====================
# Min-Max 정규화
# ====================
save_dir = os.path.join(ipynb_path, f'notebooks/test_pivot/pivotall_m_testBaseline/')

train(save_dir, x_train_m_selected, y_train, x_test_m_selected, y_test)
# ====================
# z-정규화
# ====================
save_dir = os.path.join(ipynb_path, f'notebooks/test_pivot/pivotall_z_testBaseline/')

train(save_dir, x_train_z_selected, y_train, x_test_z_selected, y_test)

Training GradientBoostingRegressor

R2 Score: 22.97

RMSE: 5315.56

NMAE: 21.0564

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.1}


Training RandomForestRegressor

R2 Score: 20.35

RMSE: 5405.00

NMAE: 21.3668

Best Hyperparameters: {'n_estimators': 200, 'max_depth': 10}




/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


Training LinearRegression

R2 Score: 19.89

RMSE: 5420.77

NMAE: 21.5579

Best Hyperparameters: {}


Training ExtraTreesRegressor

R2 Score: 12.75

RMSE: 5657.14

NMAE: 22.0695

Best Hyperparameters: {'n_estimators': 200, 'max_depth': 20}


Training AdaBoostRegressor

R2 Score: 20.20

RMSE: 5410.32

NMAE: 22.0542

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.01}




/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:53:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:53:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:53:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.



Training XGBRegressor

R2 Score: 20.27

RMSE: 5407.72

NMAE: 22.3648

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.01}




...

Training LinearRegression

R2 Score: 19.65

RMSE: 5428.97

NMAE: 21.5980

Best Hyperparameters: {}


Training ExtraTreesRegressor

R2 Score: 17.55

RMSE: 5499.17

NMAE: 21.5653

Best Hyperparameters: {'n_estimators': 200, 'max_depth': 10}


Training AdaBoostRegressor

R2 Score: 20.02

RMSE: 5416.23

NMAE: 22.1013

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.01}




/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.



Training XGBRegressor

R2 Score: 17.57

RMSE: 5498.78

NMAE: 21.4403

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.1}




/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/user/anaconda3/envs/windPower/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [21:38:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.



Training XGBRFRegressor

R2 Score: 4.35

RMSE: 5923.22

NMAE: 24.8463

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.1}


Training LGBMRegressor

R2 Score: 20.75

RMSE: 5391.70

NMAE: 22.3270

Best Hyperparameters: {'n_estimators': 100, 'learning_rate': 0.01}




In [58]:
import glob

csv_files = glob.glob(os.path.join(ipynb_path+'/notebooks/test_pivot', '**', '*.csv'), recursive=True)
print(csv_files)
total_res_df = pd.DataFrame()

for i,j in enumerate(csv_files):
    res_df=pd.read_csv(j)
    res_df['foldername'] = os.path.basename(os.path.dirname(j))
    total_res_df = pd.concat([total_res_df,res_df])
    
total_res_df.sort_values(by='NMAE', ascending=True, inplace=True)
total_res_df.head()

['/home/user/Wind_Power/notebooks/test_pivot/pivotall_m_testBaseline/model_scores.csv', '/home/user/Wind_Power/notebooks/test_pivot/pivotall_General_testBaseline/model_scores.csv', '/home/user/Wind_Power/notebooks/test_pivot/pivotall_z_testBaseline/model_scores.csv']


,Model_Name,R2_score,RMSE,NMAE,Best_Params,Predict,foldername
0,GradientBoostingRegressor,22.941394,5316.483314,21.042658,"{'n_estimators': 100, 'learning_rate': 0.1}",[12833.14796767 13293.50363905 12242.9395918 ...,pivotall_m_testBaseline
0,GradientBoostingRegressor,22.968246,5315.556924,21.056399,"{'n_estimators': 100, 'learning_rate': 0.1}",[12745.62674975 14169.66710281 13557.26514688 ...,pivotall_General_testBaseline
7,LGBMRegressor,18.803686,5457.352868,21.298523,"{'n_estimators': 100, 'learning_rate': 0.1}",[15989.9780274 15721.14518966 16312.89512624 ...,pivotall_General_testBaseline
1,RandomForestRegressor,20.581256,5397.285445,21.334361,"{'n_estimators': 200, 'max_depth': 10}",[10550.57090878 10797.67776464 9741.25228538 ...,pivotall_m_testBaseline
1,RandomForestRegressor,20.468710,5401.108394,21.359609,"{'n_estimators': 200, 'max_depth': 10}",[11042.83060205 10283.03492347 9895.30716669 ...,pivotall_z_testBaseline


In [59]:
model_path = os.path.join(ipynb_path, f'notebooks/test_pivot/pivotall_m_testBaseline/GradientBoostingRegressor.pkl')
loaded_model = joblib.load(model_path)

predicted = loaded_model.predict( x_test_z)


In [222]:
predicted.shape

(17394,)

In [61]:
import glob

csv_files = glob.glob(os.path.join(ipynb_path+'notebooks/test_pivot', '**', '*.csv'), recursive=True)
print(csv_files)
total_res_df = pd.DataFrame()

for i,j in enumerate(csv_files):
    res_df=pd.read_csv(j)
    res_df['foldername'] = os.path.basename(os.path.dirname(j))
    total_res_df = pd.concat([total_res_df,res_df])
    
total_res_df.sort_values(by='NMAE', ascending=True, inplace=True)
total_res_df.head()

[]


KeyError: 'NMAE'

In [62]:
submision_df = pd.read_csv(input_path + "submission_format.csv", encoding='CP949')
submision_df.columns

Index(['plant_name', 'end_datetime', 'period_hours', 'energy_kwh'], dtype='object')

In [74]:
# 데이터 로드
gj_test_ldaps = pd.read_parquet(input_path + "test_ldaps_gyeongju.parquet")
yg_test_ldaps = pd.read_parquet(input_path + "test_ldaps_yeonggwang.parquet")

# 데이터 파이프라인
gj_test_ldaps = DataPipeline.fit_transform(gj_test_ldaps)
yg_test_ldaps = DataPipeline.fit_transform(yg_test_ldaps)


In [75]:
gj_test_ldaps = gj_test_ldaps.pivot_table(index='dt', columns='turbine_id', aggfunc='mean')
gj_test_ldaps.columns = ['_'.join(map(str, col)) for col in gj_test_ldaps.columns]
yg_test_ldaps = yg_test_ldaps.pivot_table(index='dt', columns='turbine_id', aggfunc='mean')
yg_test_ldaps.columns = ['_'.join(map(str, col)) for col in yg_test_ldaps.columns]
####

In [76]:
gj_test_ldaps.columns = gj_test_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
gj_test_ldaps.columns = gj_test_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
gj_test_ldaps.columns = gj_test_ldaps.columns.str.replace(r'__+', '_', regex=True)
gj_test_ldaps.reset_index(inplace=True)

yg_test_ldaps.columns = yg_test_ldaps.columns.str.replace(r'[<>\[\]]', '_', regex=True)
yg_test_ldaps.columns = yg_test_ldaps.columns.str.replace(r'[^\w]', '_', regex=True)
yg_test_ldaps.columns = yg_test_ldaps.columns.str.replace(r'__+', '_', regex=True)
yg_test_ldaps.reset_index(inplace=True)

In [77]:
yg_test_ldaps = yg_test_ldaps.drop_duplicates(subset=['dt'])
gj_test_ldaps = gj_test_ldaps.drop_duplicates(subset=['dt'])


In [79]:
gj_test_ldaps.shape

(8760, 190)

In [81]:
yg_test_ldaps = yg_test_ldaps.drop_duplicates(subset=['dt'])
gj_test_ldaps = gj_test_ldaps.drop_duplicates(subset=['dt'])
model_path = os.path.join(ipynb_path, f'notebooks/test_pivot/pivotall_m_testBaseline/GradientBoostingRegressor.pkl')
loaded_model = joblib.load(model_path)



In [85]:
yg_test_ldaps, _ = addKmeansFeature(yg_test_ldaps, yg_test_ldaps.head(6).copy())
yg_test_ldaps, _ = addPCAFeature(yg_test_ldaps, yg_test_ldaps.head(6).copy())
yg_test_ldaps, _ = addKMedoidsFeature(yg_test_ldaps, yg_test_ldaps.head(6).copy())

gj_test_ldaps, _ = addKmeansFeature(gj_test_ldaps, gj_test_ldaps.head(6).copy())
gj_test_ldaps, _ = addPCAFeature(gj_test_ldaps, gj_test_ldaps.head(6).copy())
gj_test_ldaps, _ = addKMedoidsFeature(gj_test_ldaps, gj_test_ldaps.head(6).copy())

wind_speed_WTG01: PC1 설명력: 0.9992979168891907, PC2 설명력: 0.0007020226330496371
wind_speed_WTG02: PC1 설명력: 0.9992906451225281, PC2 설명력: 0.0007093080785125494
wind_speed_WTG03: PC1 설명력: 0.9992906451225281, PC2 설명력: 0.0007093080785125494
wind_speed_WTG04: PC1 설명력: 0.9992907643318176, PC2 설명력: 0.0007092328160069883
wind_speed_WTG05: PC1 설명력: 0.9992907643318176, PC2 설명력: 0.0007092328160069883
wind_speed_WTG06: PC1 설명력: 0.9992907643318176, PC2 설명력: 0.0007092328160069883
wind_speed_WTG07: PC1 설명력: 0.9992907643318176, PC2 설명력: 0.0007092328160069883
wind_speed_WTG08: PC1 설명력: 0.99929279088974, PC2 설명력: 0.0007072051521390676
wind_speed_WTG09: PC1 설명력: 0.99929279088974, PC2 설명력: 0.0007072051521390676
wind_speed_WTG10: PC1 설명력: 0.99929279088974, PC2 설명력: 0.0007072051521390676
wind_speed_WTG11: PC1 설명력: 0.9992906451225281, PC2 설명력: 0.0007093080785125494
wind_speed_WTG12: PC1 설명력: 0.9992906451225281, PC2 설명력: 0.0007093080785125494
wind_speed_WTG13: PC1 설명력: 0.9992906451225281, PC2 설명력: 0.000709308078

In [89]:
submision_df.columns

Index(['plant_name', 'end_datetime', 'period_hours', 'energy_kwh'], dtype='object')

In [98]:
# x_test_z에 'energy_kwh'와 'period_hours' 컬럼이 있는지 확인
x_test_z['energy_kwh'].max()

2.292382138115841

In [95]:
for idx, row in submision_df[submision_df['plant_name'] == '영광풍력'].iterrows():
    matching_input = yg_test_ldaps[yg_test_ldaps['dt'] == row['end_datetime']]
    
    if not matching_input.empty:
        # 필요한 컬럼이 없으면 더미 값(0)으로 추가
        if 'energy_kwh' not in matching_input.columns:
            matching_input['energy_kwh'] = 0
        if 'period_hours' not in matching_input.columns:
            matching_input['period_hours'] = 0
        
        # x_test_z와 동일한 컬럼 순서를 유지
        features = matching_input[x_test_z.columns]
        
        predicted_energy = loaded_model.predict(features)
        
        submision_df.at[idx, 'energy_kwh'] = predicted_energy[0]


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- energy_kwh
- period_hours


In [ ]:
for idx, row in submision_df[submision_df['plant_name'] == '경주풍력'].iterrows():
    matching_input = yg_test_ldaps[yg_test_ldaps['dt'] == row['end_datetime']]
    
    if not matching_input.empty:
        features = matching_input[['elevation', 'land_cover', 'surf_rough', 'frictional_vmax_50m', 'frictional_vmin_50m', 
                       'pressure', 'relative_humid', 'specific_humid', 'temp_air', 'storm_u_5m', 'storm_v_5m', 
                       'wind_u_10m', 'wind_v_10m', 'wind_speed', 'wind_direction', 'wind_speed_100m', 'cluster_4']]
        
        predicted_energy = loaded_model.predict(features)
        
        submision_df.at[idx, 'energy_kwh'] = predicted_energy[0]

In [ ]:
submision_df.to_csv(ipynb_path + '/씽씽 불어라팀 1차 중간평가 제출(모델 1).csv', index=False, encoding='CP949')